In [1]:
#tworze słownik unigramowy [słowo] : [ilość wystąpień słowa]
unigram_dict = {}
with open('./../ruznezecy/1grams.txt', 'r', encoding='utf8') as unigrams:
    for line in unigrams:
        frequency, word = line.split()
        unigram_dict[word] = frequency


In [4]:
# nowy słownik: [słowo] : [tag]
tag_dict = {}
with open('./../ruznezecy/supertags.txt', 'r', encoding='utf8') as supertags:
    for line in supertags:
        word, tag = line.split()
        tag_dict[word] = tag

In [6]:
# Tutaj przygotowuję sobie słowniki składające się z klucz(słowo) : wartość[[następnik1,częstość występowania],...
bigrams_weight_dict = {}
with open('./../ruznezecy/poleval_2grams.txt', 'r', encoding='utf8') as bigrams:
    for line in bigrams:
        current_line = line.split()
        frequency, first, second = current_line[0], current_line[1] , current_line[2]
        if int(frequency) > 5: #biorę tylko bigramy częstsze niż 5
            if first in bigrams_weight_dict.keys():
                bigrams_weight_dict[first].append([second, frequency])
            else:
                bigrams_weight_dict[first] = [[second, frequency]]         

In [7]:
UNIGRAMS_AMOUNT = 0
BIGRAMS_AMOUNT = 0
for line in unigram_dict:
   UNIGRAMS_AMOUNT += int(unigram_dict[line])

for line in bigrams_weight_dict:
    for pair in bigrams_weight_dict[line]:
        BIGRAMS_AMOUNT += int(pair[1])

In [37]:
# [tag] : [ilość wystąpień]

tag_unigram_frequency = {}
for word in tag_dict:
    word_tag = tag_dict[word]
    if word in unigram_dict.keys():
        if word_tag in tag_unigram_frequency.keys():
            tag_unigram_frequency[word_tag] += int(unigram_dict[word])
        else:
            tag_unigram_frequency[word_tag] = int(unigram_dict[word])


In [38]:
print(tag_unigram_frequency)

56, 'T1802': 13, 'T3222': 4, 'T353': 8, 'T1562': 571, 'T2808': 64, 'T1259': 8142, 'T3678': 15, 'T1279': 817, 'S107': 65, 'T299': 2996, 'T3247': 30076, 'T445': 3, 'T169': 9, 'T1917': 842, 'T2275': 32, 'T939': 87, 'T1902': 6, 'T2706': 63, 'T2521': 6367, 'T2138': 1005, 'T3026': 56, 'T3666': 108, 'T2248': 37, 'T97': 16, 'T3799': 117, 'T1134': 248, 'T69': 35, 'T163': 1, 'S94': 14, 'T942': 28, 'T0': 329, 'T1408': 756, 'T637': 3, 'T2661': 75, 'T2823': 34, 'T1': 1645, 'T1237': 206, 'T354': 30, 'T704': 1, 'T1081': 762, 'S141': 7027, 'T2186': 12, 'T3314': 5, 'T596': 111499, 'T1635': 659, 'T3699': 3, 'T416': 313, 'T2821': 16711, 'T2892': 40, 'T1577': 143, 'T3458': 514, 'S224': 775, 'T2537': 767, 'T2672': 14, 'T868': 1, 'T847': 183, 'T2699': 2019, 'T1927': 3874, 'T561': 69, 'T3065': 986, 'T2904': 1, 'T3000': 50, 'T2009': 11, 'S84': 117, 'T1355': 15, 'T2814': 275, 'T1505': 2611, 'T3809': 2945, 'T3420': 4829, 'T2293': 382, 'T948': 904, 'T406': 812, 'T3053': 2644, 'T724': 37, 'T3213': 3, 'T1843': 160

In [39]:
def end_of_word(word):
    return ('^' + word)[-3:]

def find_tag(w):  
    if w in ['.', ',', ';', ':']:
        return None
    else:
        if w in tag_dict.keys():
            return tag_dict[w]
        else:
            return None
    # for tag in tag_dict:
    #     if w in tag_dict[tag]:
    #         return tag
    # for tag in tag_dict:
    #     if end_of_word(w) == end_of_word(tag_dict[tag][0]):
    #         return tag
  

In [11]:
tag_bigram_frequency = {}
with open('./../ruznezecy/poleval_2grams.txt', 'r', encoding='utf8') as bigrams:
    for line in bigrams: 
        current_line = line.split()
        frequency, first, second = current_line[0], current_line[1] , current_line[2]
        tag_first = find_tag(first)
        tag_second = find_tag(second)
        if tag_first != None and tag_second != None:
            if tag_first in tag_bigram_frequency.keys():
                if tag_second not in tag_bigram_frequency[tag_first]:
                    tag_bigram_frequency[tag_first][tag_second] = int(frequency)
                else:
                    tag_bigram_frequency[tag_first][tag_second] += int(frequency)
            else:
                tag_bigram_frequency[tag_first] = {tag_second: int(frequency)}


': 53, 'T667': 25, 'T2693': 1757, 'T3797': 1501, 'T1875': 2602, 'T3798': 1152, 'T3407': 311, 'T1585': 611, 'T3609': 25, 'T77': 262, 'T807': 4176, 'T556': 74, 'T1633': 5479, 'T1186': 2171, 'T1698': 983, 'T46': 4488, 'T2819': 2564, 'T3046': 1405, 'T2380': 151, 'S187': 70, 'T2249': 2015, 'T2818': 101, 'T1756': 606, 'T1448': 1352, 'T3548': 169, 'T631': 2557, 'T3508': 2955, 'T51': 8718, 'T3593': 2208, 'T3531': 59, 'T3009': 115, 'T3313': 795, 'S167': 25492, 'T49': 6329, 'T2091': 675, 'T2287': 1605, 'T3765': 3981, 'T2121': 2151, 'T3513': 4554, 'T1886': 3772, 'T2978': 470, 'T1360': 1060, 'T2957': 648, 'T2010': 867, 'T1367': 1846, 'T1280': 1664, 'T1856': 182, 'T190': 213, 'T3341': 378, 'T3299': 53, 'T2305': 1315, 'T343': 163, 'T3163': 5068, 'T2959': 231, 'T472': 1238, 'T1704': 39, 'T1634': 953, 'T412': 7, 'T1188': 2031, 'T3726': 6366, 'T1895': 3370, 'T2224': 66, 'T714': 405, 'T251': 1142, 'T235': 1063, 'T3038': 2157, 'T364': 415, 'T3424': 64, 'S201': 82, 'S209': 304, 'T2223': 894, 'T314': 1512,

In [44]:
import math



def bigram_frequency(w1, w2):
    frequency = 0
    for pair in bigrams_weight_dict[w1]:
        if pair[0] == w2:
            frequency =  int(pair[1])
    return frequency

def bigram_tag_frequency(t1, t2):

    if t2 in tag_bigram_frequency[t1].keys():
        return tag_bigram_frequency[t1][t2]
    else:
        return 1
    
    

def bigram_tag_ppb(t1, t2):
    return bigram_tag_frequency(t1,t2)/BIGRAMS_AMOUNT

def bigram_ppb(w1, w2):
    return bigram_frequency(w1,w2)/BIGRAMS_AMOUNT

def unigram_ppb(w):
    if w in unigram_dict.keys():
        return int(unigram_dict[w])/UNIGRAMS_AMOUNT
    else:
        return 1

def unigram_tag_ppb(t1):
    if t1 in tag_unigram_frequency.keys():
        return int(tag_unigram_frequency[t1])/UNIGRAMS_AMOUNT
    else:
        return 1

def count_ppmi(w1, w2):
    return max(0, math.log(bigram_ppb(w1,w2) / (unigram_ppb(w1) * unigram_ppb(w2))))

def count_psm(w1, w2):
    return bigram_frequency(w1,w2) * (math.log(bigram_frequency(w1,w2)) - math.log(unigram_ppb(w1) * unigram_ppb(w2) * UNIGRAMS_AMOUNT) - 1)

def count_ppmi_with_tags(t1, t2):
    return max(0, math.log(bigram_tag_ppb(t1,t2) / (unigram_tag_ppb(t1) * unigram_tag_ppb(t2))))



In [47]:
def colocations_ppmi(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            result_dict[w + " " + second_word] = count_ppmi(w, second_word)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1

def colocations_psm(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            result_dict[w + " " + second_word] = count_psm(w, second_word)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1

def colocations_tags(w):
    result_dict = {}
    for pair in bigrams_weight_dict[w]:
        second_word = pair[0]
        if len(second_word) > 2:
            word1_tag = find_tag(w)
            word2_tag = find_tag(second_word)
            result_dict[w + " " + second_word] = count_ppmi_with_tags(word1_tag, word2_tag)
        else:
            continue
    sorted_results = {k: v for k, v in sorted(result_dict.items(), key=lambda item: item[1])}
    res = dict(reversed(list(sorted_results.items())))
    counter = 0
    for elem in res:
        if counter > 10:
            break
        else:
            print(elem, res[elem])
            counter += 1


test1 = "tata"


print("#" * 40, "TUTAJ WYNIKI PPMI:", "#" * 40)
colocations_ppmi(test1)
print("-" * 40)
colocations_ppmi("wielki")

print("#" * 40, "TUTAJ WYNIKI PSM:", "#" * 40)

colocations_psm(test1)
print("-" * 40)
colocations_psm("wielki")


print("#" * 40, "TUTAJ WYNIKI PPMI Z TAGAMI:", "#" * 40)

colocations_tags(test1)
print("-" * 40)
colocations_tags("wielki")


######################################## TUTAJ WYNIKI PPMI: ########################################
tata xenon 10.61574104980941
tata martino 9.316458065679148
tata steel 8.241243375225078
tata motors 8.01572426939775
tata kazika 6.789518034184597
tata group 5.233159130898281
tata kupił 3.9216432079310803
tata zmarł 3.7804220640919843
tata zabrał 3.7264892423918323
tata lubi 2.9950768134787213
tata pracował 2.9540832580396224
----------------------------------------
wielki penitencjariusz 11.38010682133685
wielki chingan 11.197785264542896
wielki przeorat 10.841110320604164
wielki gatsby 10.841110320604164
wielki kack 10.553428248152382
wielki szlem 10.34745250046054
wielki krzek 10.281494532668741
wielki moff 9.99381246021696
wielki szacun 9.844280726245996
wielki zderzacz 9.642414573379071
wielki mufti 9.634867367743688
######################################## TUTAJ WYNIKI PSM: ########################################
tata motors 277.4307374277651
tata steel 146.74929574389589
tata 